In [1]:
# This notebook analyzes MAANG stocks from the perspective of the typical
# Black-Scholes model using day-over-day returns.

# Using daily returns has the benefit of potentially reducing noise associated
# with seasonality etc, but more likely the lookback window used by market makers
# for example will be much shorter, say 30 days. One downside is that the
# approximation associated with Geometric Brownian Motion is less accurate.
# Intra-day (high-frequency) returns may be preferred for this reason in practice.

In [2]:
# Recall the Stochastic differential equation describing Geometric Brownian Motion
# dS = mu*S*dt + sigma*S*dW where W is a normalized Wiener process. This results
# in the S(t) = S_0 * exp((mu - sigma^2 / 2) * t + sigma * W(t)). Then we can easily
# use maximum likelihood estimation to estimate mu and sigma. It's easy to check that
# log(S_t) - log(S_0) ~ N((mu - sigma^2/2)*t, sigma^2*t).
# Then the likelihood of mu, sigma for an observed (S_{t_i}) is sum_i(log(P(S_{t_i} | S_0; mu, sigma)))
# = sum(e^())... this is kind of nasty. But it *can* be done, I just need more paper.
# Note 2 levels of exponential in some sense so exp will stay.
# = sum(1/sigma_prime*sqrt(2*pi)*e^(-1/2*(log(S_t/S_0) - (mu_prime - sigma_prime^2 / 2)t)^2 / (2*sigma_prime^2))
# where sigma_prime = sigma^2*t, mu_prime = (mu - sigma^2/2)*t.
# May need to take another log, but that's ok maximizing strictly increasing jawn is same as log^n(foo).

# sum(mu_hat * t_i) ...

# TODO(dakshs): What is the Black-Scholes MLE and why?
# Can determine sigma, but technically W_t not observed. A ver.
# Oh wait lol W_t has a damn sigma dependence. It has been a very long time since I've done any stat. Outside of memes.

In [5]:
import pandas as pd
import numpy as np

# Porting import logic from plot_log_returns.ipynb.
df = pd.read_csv('/workspaces/JumpDiffusionModeling/data/maamg_prices.csv')

In [4]:
df.head()

,MSFT_Date,MSFT_Open,MSFT_High,MSFT_Low,MSFT_Close,MSFT_Adj Close,MSFT_Volume,AMZN_Date,AMZN_Open,AMZN_High,...,GOOG_Close,GOOG_Adj Close,GOOG_Volume,AAPL_Date,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume
0,11/9/21,337.109985,338.720001,334.529999,335.950012,333.159790,21307400,11/9/21,175.762497,179.688507,...,149.248505,149.248505,16876000,11/9/21,150.199997,151.429993,150.059998,150.809998,149.939758,56787900
1,11/10/21,334.570007,334.630005,329.920013,330.799988,328.052551,25500900,11/10/21,178.193497,180.272507,...,146.626007,146.626007,22708000,11/10/21,150.020004,150.130005,147.850006,147.919998,147.066467,65187100
2,11/11/21,331.250000,333.769989,330.510010,332.429993,329.669006,16849800,11/11/21,175.649994,177.162003,...,146.748001,146.748001,12464000,11/11/21,148.960007,149.429993,147.679993,147.869995,147.016739,41000000
3,11/12/21,333.920013,337.230011,333.790009,336.720001,333.923370,23831000,11/12/21,174.250000,177.036499,...,149.645493,149.645493,17048000,11/12/21,148.429993,150.399994,147.479996,149.990005,149.124527,63804000
4,11/15/21,337.540009,337.880005,334.029999,336.070007,333.278778,16723000,11/15/21,176.850006,179.694000,...,149.388000,149.388000,16248000,11/15/21,150.369995,151.880005,149.429993,150.000000,149.134460,59222800


In [6]:
# Options on how to design this:
# 1. Plug into wolfram alpha to numerically solve the argmax lol.
# 2. Determine the MLE and just uhhh plug in for mu, sigma.
# 2 seems a a lot easier. Ok lemme just get the MLE.
# But ok so when we mean "plug in", it's really just
# mu({S_i}) = foo etc. So the difficult part ist just determining the MLE.
# And ok we can backtest on test data in a monte carlo fashion.
# Most likely a log(S_i) should be fine too as input.

In [7]:
# For simplicity, we use the closing data.
goog_log_returns = np.log((df['GOOG_Close'].iloc[1:].values/df['GOOG_Close'].iloc[:-1].values).reshape(-1,1))

from sklearn.model_selection import train_test_split

goog_train, goog_test = train_test_split(goog_log_returns, test_size = .2, random_state=0)

In [ ]:
# The derivation of Black-Scholes MLE is actually pretty involved given what I said above lol.
# See https://espace.curtin.edu.au/bitstream/handle/20.500.11937/37460/152629_28490_Misiran_Lu_Teo_%20ICOCO.pdf?sequence=2 for a reference.